In [1]:
import sys
sys.path.append("../")

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import math
from random import shuffle, choice
import copy
from utils import *
from engine import *
from IPython.display import clear_output
import time
import datetime
import re

from scipy.optimize import linprog
from collections import Counter
import itertools

In [2]:
# change inventory to sqm
# add to azure
# update front end

# TODO on algorithm
# try a heuristic split method to come up with best 
# 2 layout approach

In [98]:
# This is the memoization approach of  
# 0 / 1 Knapsack in Python in simple  
# we can say recursion + memoization = DP 
def knapsack(wt, val, W, n):     
  
    # base conditions 
    if n == 0 or W == 0: 
        return 0
    if t[n][W] != -1: 
        return t[n][W] 
  
    # choice diagram code 
    if wt[n-1] <= W: 
        t[n][W] = max( 
            val[n-1] + knapsack( 
            wt, val, W-wt[n-1], n-1),  
            knapsack(wt, val, W, n-1)) 
        return t[n][W] 
    elif wt[n-1] > W: 
        t[n][W] = knapsack(wt, val, W, n-1) 
        return t[n][W] 

def reconstruct(i, w, kp_soln, weight_of_item):
    """
    Reconstruct subset of items i with weights w. The two inputs
    i and w are taken at the point of optimality in the knapsack soln

    In this case I just assume that i is some number from a range
    0,1,2,...n
    """
    recon = set()
    # assuming our kp soln converged, we stopped at the ith item, so
    # start here and work our way backwards through all the items in
    # the list of kp solns. If an item was deemed optimal by kp, then
    # put it in our bag, otherwise skip it.
    for j in range(i)[::-1]:
        cur_val = kp_soln[j][w]
        prev_val = kp_soln[j-1][w]
        if cur_val > prev_val:
            recon.add(j)
            w = w - weight_of_item[j]
    return recon

def initt(W, n):
    # We initialize the matrix with -1 at first. 
    return [[-1 for i in range(W + 1)] for j in range(n + 1)] 

def make_best_pattern(q, w, n, usable_width=4160):
    """
    Creates the best possible pattern such that all orders are fullfilled in a single
    layout
    
    Parameters
    ----------
    q: list
        rolls required (in jumbo lengths)
    w: list
        widths required
    n: list
        neckins for widths
    usable_width: int
        jumbo/doff usable width
        
    Returns
    -------
    layout: list
        cuts for jumbo for each width (no width is excluded)
    """
    layout = [max(1, math.floor(i/sum(q)*usable_width/j)) for i,j in zip(q,w)] 

    # give priority to widths that had to round down the most
    # when filling up the rest of the pattern
    remainder = [math.remainder(i/sum(q)*usable_width/j, 1) if (math.remainder(i/sum(q)*usable_width/j, 1)
                                                        < 0) else -1 for i,j in zip(q,w) ] 
    order = np.argsort(remainder)
    while (usable_width - sum([i*j for i,j in zip(layout,w)])) > min(w):
        for i in order[::-1]:
            layout[i] += 1
            if usable_width - sum([i*j for i,j in zip(layout,w)]) < 0:
                layout[i] -= 1

    # compute the loss for the final layout
    layout_loss = usable_width - sum([i*j for i,j in zip(layout,w)])
    print("layout pattern: {}".format(dict(zip([i-j for i,j in zip(w,n)],layout))))
    print("pattern loss: {:0.2f} %".format(layout_loss/usable_width*100))

    # multiply to get the minimum doffs required
    # layout * doffs > q
    doffs = max([math.ceil(i/j) for i,j in zip(q, layout)])
    print("minimum doffs to fill order: {}".format(doffs))

    # what inventory is created
    inventory = dict(zip([i-j for i,j in zip(w,n)],[i*doffs-j for i,j in zip(layout,q)]))
    print("inventory created: {}".format(inventory))
    
    return layout

def init_layouts(B, w):
    t = []
    m = len(w)
    for i in range(m):
        pat = [0]*m
        pat[i] = -int(B/w[i])
        t.append(pat)
    return t

def store_patterns(goal=5):
    patterns = []
    bit = 0
    while len(patterns) < goal:
        found = 0
        for pair in np.argwhere(t == t.max()-bit):
            N, W = pair
            sack = reconstruct(N, W, t, s)     
            pattern = Counter(np.array(s)[list(sack)])
            loss = round((B - np.array(s)[list(sack)].sum())/B*100,2)
            patterns.append([pattern, loss])
            if len(patterns) >= goal:
                break
            found += 1
            if found > 1:
                break
        bit += 2
    return patterns

def output_results(result, lhs_ineq, B, w, n):
    sheet = np.sum([(i*j) for i,j in zip(w, np.array(lhs_ineq))],axis=0)#*np.ceil(result['x'])
    inventory = dict(zip([i-j for i,j in zip(w,n)],np.sum(np.array(lhs_ineq)*-1*np.ceil(result['x']),axis=1)-np.array(q)))

    # create layout summary
    jumbos = list(np.ceil(result['x'])[np.ceil(result['x'])>0])
    temp = np.array(lhs_ineq)*-1*np.where(np.ceil(result['x']) != 0, 1, 0)
    temp = temp[:, temp.any(0)].T
    non_zero_layouts = list([dict(zip([i-j for i,j in zip(w,n)], i)) for i in temp])

    sheet_loss = [B+i for i in sheet]
    sheet_loss = [i / B * 100 for i,j in zip(sheet_loss,np.where(result['x'] > 0, 1, 0)) if j > 0]

    # remove extra layouts due to ceiling rounding from linprog
    summary = pd.DataFrame([sheet_loss, jumbos, non_zero_layouts]).T
    summary.columns = ['loss', 'jumbos', 'layout']
    summary = summary.sort_values('loss', ascending=False).reset_index(drop=True)
    for index, layout2 in enumerate(summary['layout']):
        if all(np.array(list(inventory.values())) - np.array(list(layout2.values())) > 0):
            summary.loc[index, 'jumbos'] -= 1
            new_values = np.array(list(inventory.values())) - np.array(list(layout2.values()))
            inventory.update(zip(inventory,new_values))
    summary = summary[summary['jumbos'] != 0]

    loss = sum([i[0]*i[1] for i in summary.values])/sum([i[1] for i in summary.values])
    sqm_inventory = np.sum([i*j*.001*L for i,j in zip (inventory.keys(),inventory.values())])
    sqm_produced = np.sum(jumbos)*L*B*.001
    sqm_loss = sqm_produced*loss/100

    print("total loss:      {:0.2f} % ({:.2e} sqm)".format(loss, sqm_loss))
    print("total inventory: {:.2f} % ({:.2e} sqm)".format(sqm_inventory/sqm_produced*100, sqm_inventory), end = '\n\n')
    print("inventory created: {}".format(inventory), end = '\n\n')
    # print("total inventory rolls: {:n} ({:.2e} sqm)".format(sum(list(inventory.values())), sqm_inventory), end='\n\n')
    print("layout summary:", end = '\n\n')
    for i in summary.values:
        print("loss: {:.2f}% \t {} x\t {}".format(i[0], i[1], i[2]))
    print('')
    print("total jumbos: {} ({:.2e} sqm)".format(np.sum(summary['jumbos']), sqm_produced))
    
    return loss, inventory, summary

# choose max unique widths per doff
def find_optimum(patterns, layout, max_patterns = 3, prioritize = 'time'):
    if prioritize == 'material loss':
        inv_loss = 0
    elif prioritize == 'time':
        inv_loss = 1
    if 1 < max_patterns < 4:
        # find best of X combination
        if len(w) <= max_combinations:
            pattern_combos = list(itertools.combinations(patterns,r=max_patterns-1))
        else:
            pattern_combos = list(itertools.combinations(patterns,r=max_patterns))
        print("{} possible max {} patterns".format(len(pattern_combos),max_patterns), end='\n\n')
        best_of = []
        for combo in pattern_combos:
            patterns2 = combo
            lhs_ineq = []
            for pattern in patterns2:
                inset = []
                for width in w:
                    try:
                        inset.append(-pattern[0][width])
                    except:
                        inset.append(0)
                lhs_ineq.append(inset)
        #     naive = init_layouts(B, w)
        #     lhs_ineq = lhs_ineq + naive
            if len(w) <= max_combinations:
                lhs_ineq.append([-i for i in layout])
            lhs_ineq = np.array(lhs_ineq).T.tolist()
            rhs_ineq = [-i for i in q]
            obj = np.ones(len(lhs_ineq[0]))

            result = linprog(c=obj, 
                    A_ub=lhs_ineq,
                    b_ub=rhs_ineq, 
                    method="revised simplex")
            if result['success'] == True:
                sheet = np.sum([(i*j) for i,j in zip(w, np.array(lhs_ineq))],axis=0)#*np.ceil(result['x'])
                inventory = dict(zip([i-j for i,j in zip(w,n)],np.sum(np.array(lhs_ineq)*-1*\
                                                                      np.ceil(result['x']),axis=1)-np.array(q)))

                # create layout summary
                jumbos = list(np.ceil(result['x'])[np.ceil(result['x'])>0])
                temp = np.array(lhs_ineq)*-1*np.where(np.ceil(result['x']) != 0, 1, 0)
                temp = temp[:, temp.any(0)].T
                non_zero_layouts = list([dict(zip([i-j for i,j in zip(w,n)], i)) for i in temp])

                sheet_loss = [B+i for i in sheet]
                sheet_loss = [i / B * 100 for i,j in zip(sheet_loss,np.where(result['x'] > 0, 1, 0)) if j > 0]

                # remove extra layouts due to ceiling rounding from linprog
                sorted_jumbos = [x for _,x in sorted(zip(sheet_loss,jumbos))][::-1]
                sorted_layouts = np.array(non_zero_layouts)[np.array(sheet_loss).argsort()][::-1]
                sorted_losses = [x for _,x in sorted(zip(sheet_loss,sheet_loss))][::-1]
                for index, layout2 in enumerate(np.array(non_zero_layouts)[np.array(sheet_loss).argsort()][::-1]):
                    if all(np.array(list(inventory.values())) - np.array(list(layout2.values())) > 0):
                        sorted_jumbos[index] -= 1
                        new_values = np.array(list(inventory.values())) - np.array(list(layout2.values()))
                        inventory.update(zip(inventory,new_values))

                        # clear layouts that have been set to 0
                summary = (list(zip(sorted_jumbos, sorted_layouts, sorted_losses)))
                summ = []
                for i in summary:
                    if i[0] > 0:
                        summ.append(i)
                summary=summ
                loss = sum([i[0]*i[2] for i in summary])/sum([i[0] for i in summary])

                best_of.append([loss, sum(list(inventory.values())), patterns2])

        # minimize inventory or minimize mat. loss
        arr = np.array(best_of)
        patterns_final = arr[np.argmin(arr[:,inv_loss])][2]
    elif max_patterns == 1:
        patterns_final = [[dict(zip(w,layout)), 0]]

    else:
        patterns_final = patterns

    # find overall best combination
    # format layouts for linear optimization
    lhs_ineq = []
    for pattern in patterns_final:
        inset = []
        for width in w:
            try:
                inset.append(-pattern[0][width])
            except:
                inset.append(0)
        lhs_ineq.append(inset)
    # naive = init_layouts(B, w)
    # lhs_ineq = lhs_ineq + naive
    if len(w) <= max_combinations:
        lhs_ineq.append([-i for i in layout])
    lhs_ineq = np.array(lhs_ineq).T.tolist()
    rhs_ineq = [-i for i in q]
    obj = np.ones(len(lhs_ineq[0]))

    result = linprog(c=obj, 
            A_ub=lhs_ineq,
            b_ub=rhs_ineq, 
            method="revised simplex")

    return output_results(result, lhs_ineq, B, w, n)

In [101]:
df = load_schedule(
    customer='AHP',
    technology='SM',
    color='WHITE',
    cycle='CYCLE 2',
)
# df = df.iloc[:10]
B = 4160
w, q, L, n = process_schedule(
    df,
    B=B,
    put_up=17000,
    doffs_in_jumbo=6,
    verbiose=True,
)

The important variables

widths: [160, 195, 205, 210, 220, 235] (mm)
neck in: [4, 5, 5, 7, 7, 7] (mm)
jumbo length (L): 102000 (m)
undeckled jumbos needed (q): [71, 242, 108, 265, 141, 30]


In [102]:
layout = make_best_pattern(q, w, n)
max_combinations = 3

combos = list(itertools.combinations(w,r=max_combinations))
print('')
print("{} possible max {} combinations".format(len(combos),max_combinations))
patterns = []
for combo in combos:
    
    # only provide knapsack with relevant variables
    s = []
    for i in combo:
        s += (int(B/i)*[i])
    t = initt(B,len(s))
    knapsack(s, s, B, len(s))
    t = np.array(t)
    patterns += store_patterns(goal=5)
uni_list = []
for i in patterns:
    if i not in uni_list:
        uni_list.append(i)
patterns = uni_list
patterns = list(np.array(patterns)[np.array(patterns)[:,1]>=0])
print("{} unique patterns found".format(len(patterns)))

layout pattern: {160: 3, 195: 5, 205: 2, 210: 6, 220: 3, 235: 1}
pattern loss: 0.55 %
minimum doffs to fill order: 54
inventory created: {160: 91, 195: 28, 205: 0, 210: 59, 220: 21, 235: 24}

20 possible max 3 combinations
55 unique patterns found


In [124]:
loss, inventory, summary = find_optimum(patterns, layout, max_patterns=3, prioritize='time')

26235 possible max 3 patterns

total loss:      0.08 % (1.64e+04 sqm)
total inventory: 9.78 % (1.99e+06 sqm)

inventory created: {160: 59.0, 195: 11.0, 205: 27.0, 210: 2.0, 220: 9.0, 235: 0.0}

layout summary:

loss: 0.17% 	 23.0 x	 {160: 0, 195: 11, 205: 0, 210: 9, 220: 0, 235: 0}
loss: 0.00% 	 10.0 x	 {160: 13, 195: 0, 205: 0, 210: 6, 220: 0, 235: 3}
loss: 0.00% 	 15.0 x	 {160: 0, 195: 0, 205: 9, 210: 0, 220: 10, 235: 0}

total jumbos: 48.0 (2.04e+07 sqm)


# sqm produced/inventory

In [77]:

max_patterns = 3
prioritize = 'time'
if prioritize == 'material loss':
    inv_loss = 0
elif prioritize == 'time':
    inv_loss = 1
if 1 < max_patterns < 4:
    # find best of X combination
    if len(w) <= max_combinations:
        pattern_combos = list(itertools.combinations(patterns,r=max_patterns-1))
    else:
        pattern_combos = list(itertools.combinations(patterns,r=max_patterns))
    print("{} possible max {} patterns".format(len(pattern_combos),max_patterns), end='\n\n')
    best_of = []
    for combo in pattern_combos:
        patterns2 = combo
        lhs_ineq = []
        for pattern in patterns2:
            inset = []
            for width in w:
                try:
                    inset.append(-pattern[0][width])
                except:
                    inset.append(0)
            lhs_ineq.append(inset)
    #     naive = init_layouts(B, w)
    #     lhs_ineq = lhs_ineq + naive
        if len(w) <= max_combinations:
            lhs_ineq.append([-i for i in layout])
        lhs_ineq = np.array(lhs_ineq).T.tolist()
        rhs_ineq = [-i for i in q]
        obj = np.ones(len(lhs_ineq[0]))

        result = linprog(c=obj, 
                A_ub=lhs_ineq,
                b_ub=rhs_ineq, 
                method="revised simplex")
        if result['success'] == True:
            sheet = np.sum([(i*j) for i,j in zip(w, np.array(lhs_ineq))],axis=0)#*np.ceil(result['x'])
            inventory = dict(zip([i-j for i,j in zip(w,n)],np.sum(np.array(lhs_ineq)*-1*\
                                                                  np.ceil(result['x']),axis=1)-np.array(q)))

            # create layout summary
            jumbos = list(np.ceil(result['x'])[np.ceil(result['x'])>0])
            temp = np.array(lhs_ineq)*-1*np.where(np.ceil(result['x']) != 0, 1, 0)
            temp = temp[:, temp.any(0)].T
            non_zero_layouts = list([dict(zip([i-j for i,j in zip(w,n)], i)) for i in temp])

            sheet_loss = [B+i for i in sheet]
            sheet_loss = [i / B * 100 for i,j in zip(sheet_loss,np.where(result['x'] > 0, 1, 0)) if j > 0]

            # remove extra layouts due to ceiling rounding from linprog
            sorted_jumbos = [x for _,x in sorted(zip(sheet_loss,jumbos))][::-1]
            sorted_layouts = np.array(non_zero_layouts)[np.array(sheet_loss).argsort()][::-1]
            sorted_losses = [x for _,x in sorted(zip(sheet_loss,sheet_loss))][::-1]
            for index, layout2 in enumerate(np.array(non_zero_layouts)[np.array(sheet_loss).argsort()][::-1]):
                if all(np.array(list(inventory.values())) - np.array(list(layout2.values())) > 0):
                    sorted_jumbos[index] -= 1
                    new_values = np.array(list(inventory.values())) - np.array(list(layout2.values()))
                    inventory.update(zip(inventory,new_values))

                    # clear layouts that have been set to 0
            summary = (list(zip(sorted_jumbos, sorted_layouts, sorted_losses)))
            summ = []
            for i in summary:
                if i[0] > 0:
                    summ.append(i)
            summary=summ
            loss = sum([i[0]*i[2] for i in summary])/sum([i[0] for i in summary])

            best_of.append([loss, sum(list(inventory.values())), patterns2])

    # minimize inventory or minimize mat. loss
    arr = np.array(best_of)
    patterns_final = arr[np.argmin(arr[:,inv_loss])][2]
elif max_patterns == 1:
    patterns_final = [[dict(zip(w,layout)), 0]]

else:
    patterns_final = patterns

# find overall best combination
# format layouts for linear optimization
lhs_ineq = []
for pattern in patterns_final:
    inset = []
    for width in w:
        try:
            inset.append(-pattern[0][width])
        except:
            inset.append(0)
    lhs_ineq.append(inset)
# naive = init_layouts(B, w)
# lhs_ineq = lhs_ineq + naive
if len(w) <= max_combinations:
    lhs_ineq.append([-i for i in layout])
lhs_ineq = np.array(lhs_ineq).T.tolist()
rhs_ineq = [-i for i in q]
obj = np.ones(len(lhs_ineq[0]))

result = linprog(c=obj, 
        A_ub=lhs_ineq,
        b_ub=rhs_ineq, 
        method="revised simplex")

4060 possible max 3 patterns



In [97]:

sheet = np.sum([(i*j) for i,j in zip(w, np.array(lhs_ineq))],axis=0)#*np.ceil(result['x'])
inventory = dict(zip([i-j for i,j in zip(w,n)],np.sum(np.array(lhs_ineq)*-1*np.ceil(result['x']),axis=1)-np.array(q)))

# create layout summary
jumbos = list(np.ceil(result['x'])[np.ceil(result['x'])>0])
temp = np.array(lhs_ineq)*-1*np.where(np.ceil(result['x']) != 0, 1, 0)
temp = temp[:, temp.any(0)].T
non_zero_layouts = list([dict(zip([i-j for i,j in zip(w,n)], i)) for i in temp])

sheet_loss = [B+i for i in sheet]
sheet_loss = [i / B * 100 for i,j in zip(sheet_loss,np.where(result['x'] > 0, 1, 0)) if j > 0]

# remove extra layouts due to ceiling rounding from linprog
summary = pd.DataFrame([sheet_loss, jumbos, non_zero_layouts]).T
summary.columns = ['loss', 'jumbos', 'layout']
summary = summary.sort_values('loss', ascending=False).reset_index(drop=True)
for index, layout2 in enumerate(summary['layout']):
    if all(np.array(list(inventory.values())) - np.array(list(layout2.values())) > 0):
        summary.loc[index, 'jumbos'] -= 1
        new_values = np.array(list(inventory.values())) - np.array(list(layout2.values()))
        inventory.update(zip(inventory,new_values))
summary = summary[summary['jumbos'] != 0]

loss = sum([i[0]*i[1] for i in summary.values])/sum([i[1] for i in summary.values])
sqm_inventory = np.sum([i*j*.001*L for i,j in zip (inventory.keys(),inventory.values())])
sqm_produced = np.sum(jumbos)*L*B*.001
sqm_loss = sqm_produced*loss/100

print("total loss:      {:0.2f} % ({:.2e} sqm)".format(loss, sqm_loss))
print("total inventory: {:.2f} % ({:.2e} sqm)".format(sqm_inventory/sqm_produced*100, sqm_inventory), end = '\n\n')
print("inventory created: {}".format(inventory), end = '\n\n')
# print("total inventory rolls: {:n} ({:.2e} sqm)".format(sum(list(inventory.values())), sqm_inventory), end='\n\n')
print("layout summary:", end = '\n\n')
for i in summary.values:
    print("loss: {:.2f}% \t {} x\t {}".format(i[0], i[1], i[2]))
print('')
print("total jumbos: {} ({:.2e} sqm)".format(np.sum(summary['jumbos']), sqm_produced))

total loss:      0.23 % (4.46e+04 sqm)
total inventory: 6.00 % (1.15e+06 sqm)

inventory created: {160: 2.0, 195: 6.0, 205: 8.0, 220: 8.0, 235: 27.0}

layout summary:

loss: 0.48% 	 14.0 x	 {160: 0, 195: 6, 205: 14, 220: 0, 235: 0}
loss: 0.14% 	 22.0 x	 {160: 0, 195: 7, 205: 0, 220: 10, 235: 2}
loss: 0.05% 	 8.0 x	 {160: 7, 195: 14, 205: 1, 220: 0, 235: 0}

total jumbos: 44.0 (1.91e+07 sqm)


In [79]:
# this is sqm inventory by key/values in inventory dict
sqm_inventory = np.sum([i*j*.001*L for i,j in zip (inventory.keys(),inventory.values())])
print("{:.2e}".format(sqm_inventory))

1.15e+06


In [89]:
# total produced
sqm_produced = np.sum(jumbos)*L*B*.001
print("{:.2e}".format(sqm_produced))

1.91e+07


In [94]:
sqm_inventory/sqm_produced*100

6.001602564102564

In [92]:
loss*sqm_produced

4464818.181818182

In [96]:
loss

0.23382867132867133

In [83]:
loss

0.23382867132867133

In [95]:
sqm_loss = sqm_produced*loss/100
print("{:.2e}".format(sqm_loss))

4.46e+04


# heuristic 2-layout seed, max 3 widths

In [156]:
# for make best pattern
# need usable width
# q, w, n
# if there are 4+ widths
# need to split them up
# which width is split across
# layouts? probably the one that has the greatest LM

In [157]:
w

[158, 175, 215, 225, 241]

In [158]:
q

[32, 109, 75, 22, 460]

In [160]:
usable_width = 4160
layout = [max(1, math.floor(i/sum(q)*usable_width/j)) for i,j in zip(q,w)]

In [174]:
[[]]*len(w)

[[], [], [], [], []]

In [225]:
print(layout) # we have this ideal layout, can we use it to split across two layouts?

# for 5 unique widths with 3 max: 2-1, 2-1
# for 5 unique widths with 4 max: 1-3, 1-3
# for 5 unique widths with 2 max: no soln
# ok so seed two layouts
max_combinations = 3

if max_combinations == 3 and len(w) == 5:
    layout1 = [0]*len(w)
    layout2 = [0]*len(w)
    
    split = np.argmax(layout) # we'll split the most common width across both
    if layout[split]%2 != 0:
        layout1[split] = math.floor(layout[split]/2)
        layout2[split] = math.ceil(layout[split]/2)
    else:
        layout2[split] = layout1[split] = layout[split]/2

while (usable_width - sum([i*j for i,j in zip(layout1,w)])) > min(w):
    remainder = [math.remainder(i/sum(q)*usable_width/j, 1) if (math.remainder(i/sum(q)*usable_width/j, 1)
                                                    < 0) else -1 for i,j in zip(q,w) ] 
    i = np.argsort(remainder)[::-1][0]
#     for i in order[::-1]:
    layout1[i] += 1
    if usable_width - sum([i*j for i,j in zip(layout1,w)]) < 0:
        layout1[i] -= 1
layout1

[2, 3, 2, 1, 11]


[18, 0, 0, 0, 5]

In [232]:
[math.remainder(i/sum(q)*usable_width/j, 1)  for i,j in zip(q,w) ]

[0.20706539479888275,
 -0.2878428162095781,
 0.0790297860998197,
 -0.4172556510665393,
 0.3757148462114639]

In [233]:
[i/sum(q)*usable_width/j for i,j in zip(q,w)]

[1.2070653947988828,
 3.712157183790422,
 2.0790297860998197,
 0.5827443489334607,
 11.375714846211464]

In [226]:
print(w)
print(q)

[158, 175, 215, 225, 241]
[32, 109, 75, 22, 460]


In [220]:
layout2

[0, 0, 0, 0, 6]

In [164]:
# give priority to widths that had to round down the most
# when filling up the rest of the pattern
remainder = [math.remainder(i/sum(q)*usable_width/j, 1) if (math.remainder(i/sum(q)*usable_width/j, 1)
                                                    < 0) else -1 for i,j in zip(q,w) ] 
order = np.argsort(remainder)
order

array([0, 2, 4, 3, 1])

In [222]:
layout1

[2, 3, 3, 3, 8]

In [223]:
while (usable_width - sum([i*j for i,j in zip(layout1,w)])) > min(w):
    remainder = [math.remainder(i/sum(q)*usable_width/j, 1) if (math.remainder(i/sum(q)*usable_width/j, 1)
                                                    < 0) else -1 for i,j in zip(q,w) ] 
    i = np.argsort(remainder)[::-1][0]
#     for i in order[::-1]:
    layout1[i] += 1
    if usable_width - sum([i*j for i,j in zip(layout1,w)]) < 0:
        layout1[i] -= 1
layout1

In [ ]:



while (usable_width - sum([i*j for i,j in zip(layout,w)])) > min(w):
    for i in order[::-1]:
        layout[i] += 1
        if usable_width - sum([i*j for i,j in zip(layout,w)]) < 0:
            layout[i] -= 1

# compute the loss for the final layout
layout_loss = usable_width - sum([i*j for i,j in zip(layout,w)])
print("layout pattern: {}".format(dict(zip([i-j for i,j in zip(w,n)],layout))))
print("pattern loss: {:0.2f} %".format(layout_loss/usable_width*100))

# multiply to get the minimum doffs required
# layout * doffs > q
doffs = max([math.ceil(i/j) for i,j in zip(q, layout)])
print("minimum doffs to fill order: {}".format(doffs))

# what inventory is created
inventory = dict(zip([i-j for i,j in zip(w,n)],[i*doffs-j for i,j in zip(layout,q)]))
print("inventory created: {}".format(inventory))

13244 possible max 3 patterns

total loss: 0.12 %

inventory created: {154: 200.0, 170: 3.0, 208: 2.0, 218: 20.0, 234: 9.0}

total inventory rolls: 234

layout summary:

loss: 0.24% 	 7.0 x	 {154: 0, 170: 16, 208: 0, 218: 6, 234: 0}
loss: 0.10% 	 29.0 x	 {154: 0, 170: 0, 208: 7, 218: 0, 234: 11}
loss: 0.10% 	 11.0 x	 {154: 8, 170: 0, 208: 0, 218: 0, 234: 12}

total layouts: 47.0


(0.11763502454991817,
 {154: 200.0, 170: 3.0, 208: 2.0, 218: 20.0, 234: 9.0},
 [(7.0, {154: 0, 170: 16, 208: 0, 218: 6, 234: 0}, 0.2403846153846154),
  (29.0, {154: 0, 170: 0, 208: 7, 218: 0, 234: 11}, 0.09615384615384616),
  (11.0, {154: 8, 170: 0, 208: 0, 218: 0, 234: 12}, 0.09615384615384616)])

In [155]:
loss, inventory, summary = find_optimum(patterns, max_patterns=4, prioritize='time')

total loss: 0.31 %

inventory created: {154: 52.0, 170: 17.0, 208: 9.0, 218: 20.0, 234: 2.0}

total inventory rolls: 100

layout summary:

loss: 0.31% 	 42.0 x	 {154: 2, 170: 3, 208: 2, 218: 1, 234: 11}

total layouts: 42.0
